In [1]:
from pathlib import Path
import pandas as pd

import os
from pathlib import Path
import pandas as pd
import datetime
from datetime import datetime


def import_data_from_csv() -> dict:
    """Read data from csv files (prepared by transfermrkt dataset)

    Returns:
        list[pd.DataFrame] : list of dataframes (single csv to single dataframe)
    """

    # NOTE: This wont work for jupyter so we are using sth else for now
    # # Get the absolute path of the current script (jupyter dir)
    # BASE_DIR = Path(__file__).resolve().parent
        
    # NOTE: Later when we are converting this into an actual python file, comment this line  
    # Define the base and data directories
    BASE_DIR = Path.cwd()
    DATA_DIR = BASE_DIR.parents[0] / "data" / "transfer_data"
    # CLUBS_FILE = "clubs.csv"
    # ELOS_FILE = "club_elo_2024-09-21.csv"
    # # Load CSV data
    # # fbref_df = pd.read_csv(os.path.join(DATA_DIR, FBREF_FILE), sep=',', encoding='UTF-8')
    # clubs_df = pd.read_csv(os.path.join(DATA_DIR, CLUBS_FILE), sep=",", encoding="UTF-8")
    # elos_df = pd.read_csv(os.path.join(DATA_DIR, ELOS_FILE), sep=",", encoding="UTF-8")
    # elos_df = elos_df.head(632)  # Restrict the ELO data to the first 632 rows

    # # Build the path to the data directory
    # DATA_DIR = BASE_DIR / "transfer_data"

    # BASE_DIR = Path.cwd()

    # Build the path to the data directory

    # # Example of using the path to the data directory
    # csv_file = DATA_DIR / 'your_file.csv'
    # print(csv_file)

    # import all files in Data folder and read into dataframes
    dataframes = {}

    

    # Actual reading csv flies
    for dirpath, dirname, filenames in os.walk(DATA_DIR):
        for filename in filenames:
            file = filename.split(".")
            file = file[0] + "_df"
            if file != "_df":
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, sep=",", encoding="UTF-8")
                exec(f"{file} = df.copy()")
                print(file, df.shape)
                dataframes[file] = df.copy()
    print("Data imported")

    return dataframes



In [2]:
# Below is just to suppress warnings...
competitions_df = pd.DataFrame
appearances_df = pd.DataFrame
player_valuations_df = pd.DataFrame
game_events_df = pd.DataFrame
players_df = pd.DataFrame
games_df = pd.DataFrame
club_games_df = pd.DataFrame
clubs_df = pd.DataFrame
dataframes = import_data_from_csv()

for file_name, dataframe in dataframes.items():
    exec(f"{file_name} = dataframe.copy()")
    # print(file_name)
    # print(dataframe)

game_lineups_df (2191911, 10)
competitions_df (44, 11)
appearances_df (1610495, 13)
player_valuations_df (486009, 5)
game_events_df (761513, 10)
transfers_df (77585, 10)
players_df (32392, 23)
games_df (69635, 23)
club_games_df (139270, 11)
clubs_df (439, 17)
Data imported


In [12]:
def add_season_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean up date to datetime datatype
    Add month_year, season column based on date
    @param df: 
    @return: 
    """
    # Clean up player valuations
    df_copy = df.copy()
    df_copy['date'] = pd.to_datetime(df_copy['date'])
    df_copy['month_year'] = df_copy['date'].apply(lambda x: f"08 {x.year}" if x >= pd.Timestamp(x.year, 7, 1)\
        else f"08 {x.year-1}")
    df_copy['season'] = df_copy['date'].apply(lambda x: f"{x.year}-{x.year+1}" if x >= pd.Timestamp(x.year, 7, 1) \
        else f"{x.year-1}-{x.year}")
    # df_copy = df_copy.loc[df_copy.groupby(['season'])['date'].idxmin()] #Get value only at start of season
    return df_copy
    

In [13]:
# Clean up player valuations
# player_valuations_df['date'] = pd.to_datetime(player_valuations_df['date'])
# player_valuations_df['month_year'] = player_valuations_df['date'].apply(lambda x: f"08 {x.year}" \
#     if x >= pd.Timestamp(x.year, 7, 1) else f"08 {x.year-1}")
# player_valuations_df['season'] = player_valuations_df['date'].apply(lambda x: f"{x.year}-{x.year+1}" \
#     if x >= pd.Timestamp(x.year, 7, 1) else f"{x.year-1}-{x.year}")
# player_valuations_df = player_valuations_df.loc[player_valuations_df.groupby(['player_id','season'])['date'].idxmin()] #Get value only at start of season
df_sorted = add_season_column(player_valuations_df)
df_sorted = df_sorted.loc[df_sorted.groupby(['player_id','season'])['date'].idxmin()] #Get value only at start of season
df_sorted.head()

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,month_year,season
4,10,2004-10-04,7000000,398,IT1,08 2004,2004-2005
4017,10,2005-09-30,15000000,398,IT1,08 2005,2005-2006
6254,10,2006-07-15,30000000,398,IT1,08 2006,2006-2007
16465,10,2008-06-04,20000000,398,IT1,08 2007,2007-2008
26702,10,2009-06-10,18000000,398,IT1,08 2008,2008-2009


,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,month_year,season
4,10,2004-10-04,7000000,398,IT1,08 2004,2004-2005
4017,10,2005-09-30,15000000,398,IT1,08 2005,2005-2006
6254,10,2006-07-15,30000000,398,IT1,08 2006,2006-2007
16465,10,2008-06-04,20000000,398,IT1,08 2007,2007-2008
26702,10,2009-06-10,18000000,398,IT1,08 2008,2008-2009
...,...,...,...,...,...,...,...
484243,1240762,2024-06-10,50000,2778,DK1,08 2023,2023-2024
480539,1247360,2024-06-03,50000,3524,FR1,08 2023,2023-2024
485876,1263365,2024-09-12,150000,48332,UKR1,08 2024,2024-2025
485877,1290268,2024-09-12,150000,6992,UKR1,08 2024,2024-2025


In [4]:
# From the oldest to most recent, init. player's elo based on market value.
df_sorted = player_valuations_df.sort_values(by=['player_id', 'date'])
oldest_player_valuations_df = df_sorted.drop_duplicates(subset='player_id', keep='first')


In [5]:
df = oldest_player_valuations_df.copy()
seasons = sorted(oldest_player_valuations_df['season'].unique())
seasons

['1999-2000',
 '2001-2002',
 '2003-2004',
 '2004-2005',
 '2005-2006',
 '2006-2007',
 '2007-2008',
 '2008-2009',
 '2009-2010',
 '2010-2011',
 '2011-2012',
 '2012-2013',
 '2013-2014',
 '2014-2015',
 '2015-2016',
 '2016-2017',
 '2017-2018',
 '2018-2019',
 '2019-2020',
 '2020-2021',
 '2021-2022',
 '2022-2023',
 '2023-2024',
 '2024-2025']

In [13]:
players_df.head()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [ ]:
# df = players_df.copy()
# for index, row in df.iterrows():
#     df['elo'] = 


In [17]:
def sort_df_by_date(df: pd.DataFrame) -> pd.DataFrame:
    """Sort dataframe by date
    It handles changing the date to datetime format as well

    Args:
        df (pd.DataFrame): Assume club_games_df have "date"

    Returns:
        pd.DataFrame: _description_
    """

    # NOTE: This is why it might fail? Chagne to index or columns if this fails
    if "date" not in df.columns:
        raise ValueError("The 'date' column does not exist in the DataFrame.")

    df["date"] = df["date"].str.strip()
    df["date"] = pd.to_datetime(df["date"], errors="coerce", format="%Y-%m-%d")
    df.dropna(subset=["date"], inplace=True)
    # Sort by date
    df = df.sort_values(by=["date"])

    return df


games_df = sort_df_by_date(games_df)


,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
14610,2234412,ELQ,2012,First Round 1st leg,2012-07-03,24832,10694,0,6,NaN,...,Gundadalur,650.0,Andris Treimanis,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:6,international_cup
14605,2231979,CLQ,2012,First Round 1st leg,2012-07-03,3471,10532,0,0,NaN,...,Windsor Park,1341.0,Wim Smet,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
43075,2231978,CLQ,2012,First Round 1st leg,2012-07-03,853,10747,7,0,NaN,...,Stade Jos Nosbaum,1070.0,Alan Mario Sant,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,7:0,international_cup
10890,2235764,ELQ,2012,First Round 1st leg,2012-07-03,1167,5594,0,0,NaN,...,Milano Arena,2000.0,Eitan Shmuelevic,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
46883,2231982,CLQ,2012,First Round 1st leg,2012-07-03,6335,28958,8,0,NaN,...,Tony Bezzina Stadium,1136.0,Johnny Casanova,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,8:0,international_cup


In [44]:
ronaldo = players_df.loc[players_df['name'].str.contains('Cristiano Ronaldo')]
# ronaldo
# def get_appearnces_list(game_id):
#     return res_df 
# # appearances_df.tail(5)
# get_appearnces_list(4451012)

    # res_df = appearances_df.loc[appearances_df['appearance_id'].str.game_id]
# appeared_players_df = appearances_df.loc[appearances_df['game_id'] == game_id]
def init_player_elo(player_id, game_id):
    # 1. Get player's club (at that time) and check if we have enough elo data of that club

    # Note, that, since we chronologically loop through games, his teammate's elo will be elo of that time!
    # However, be careful finding out which club he was playing for at THAT time.
    pass

def get_player_elo(player_id, game_id):
    """
    Get player {player_id} elo, and if elo is not initialised, it will initialise the player's elo.
    @param player_id: player's id
    @return: player['elo'], integer
    """
    player: pd.DataFrame = players_df.loc[players_df['player_id'] == player_id]
    # If there is multiple....
    if len(player) > 1:
        raise ValueError(f"Multiple results found for player {player_id}. Expected only one")
    elif len(player) == 0:
        raise ValueError(f"No results found for player {player_id}")
    
    # Nothing went wrong
    if 'elo' in player.columns && player['elo'] is not None:
        # Elo alr exist
        return player['elo']
    else:
        # Elo doesn't exist
        return init_player_elo(player_id, game_id)
        
    

def calculate_player_elo(appearances_df, game_id):
    appeared_players_df: pd.DataFrame = appearances_df.loc[appearances_df['game_id'] == game_id]
    for index, column in appeared_players_df.iterrows():
        # Get list of players who appeared in certain game. 
        player = players_df.loc[players_df['player_id'] == column['player_id']]
        # Get or init. player elo
        player['elo']
        print(player['name'])
    
calculate_player_elo(appearances_df, 2235545)
appearances_df

1637    Willem Janssen
Name: name, dtype: object
2522    Wout Brama
Name: name, dtype: object
2559    Robbert Schilder
Name: name, dtype: object
2810    Wesley Verhoek
Name: name, dtype: object
3037    Dusan Tadic
Name: name, dtype: object
365    Peter Wisgerhof
Name: name, dtype: object
372    Sander Boschker
Name: name, dtype: object
4142    Nils Röseler
Name: name, dtype: object
5320    Nacer Chadli
Name: name, dtype: object
5379    Joshua John
Name: name, dtype: object
5668    Leroy Fer
Name: name, dtype: object
5669    Tim Breukers
Name: name, dtype: object
6692    Glynor Plet
Name: name, dtype: object
6868    Nicky Kuiper
Name: name, dtype: object


,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45
3,2234418_73333,2234418,73333,1274,6646,2012-07-05,Vegar Hedenstad,ELQ,0,0,0,0,90
4,2234421_122011,2234421,122011,195,3008,2012-07-05,Markus Henriksen,ELQ,0,0,0,1,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610490,4451012_617754,4451012,617754,1053,1053,2024-09-19,Mylian Jiménez,DKP,0,0,0,0,90
1610491,4451012_627208,4451012,627208,1053,1053,2024-09-19,Andres Jasson,DKP,0,0,0,0,23
1610492,4451012_634848,4451012,634848,1053,1053,2024-09-19,Jubril Adedeji,DKP,1,0,0,1,23
1610493,4451012_666563,4451012,666563,1053,1053,2024-09-19,Kelvin John,DKP,0,0,2,0,90


In [ ]:
# 
# 
# def sort_games_by_date(df: pd.DataFrame) -> pd.DataFrame:
#     """Sort club_games_df by date
#     It handles changing the date to datetime format as well
# 
#     Args:
#         club_games_df (pd.DataFrame): Assume club_games_df have "date"
# 
#     Returns:
#         pd.DataFrame: _description_
#     """
# 
#     # NOTE: This is why it might fail? Chagne to index or columns if this fails
#     if "date" not in df.columns:
#         raise ValueError("The 'date' column does not exist in the DataFrame.")
# 
#     df["date"] = df["date"].str.strip()
#     df["date"] = pd.to_datetime(df["date"], errors="coerce", format="%Y-%m-%d")
#     df.dropna(subset=["date"], inplace=True)
#     # Sort by date
#     df = df.sort_values(by=["date"])
# 
#     return df
# 
# 
# def expected_chance_of_success(home_team_elo, away_team_elo) -> int:
#     """Calculate chance of success E_home
# 
#     Args:
#         home_team_elo (int): _description_
#         away_team_elo (int): _description_
# 
#     Returns:
#         int : Expected chance of home team winning
#     """
#     return 1 / (1 + 10 ** ((away_team_elo - home_team_elo) / 400))
# 
# 
# # def decide_K_value(game: pd.Series):
# #     """Decide K value based on league or etc?
# 
# #     Args:
# #         game (pd.Series): _description_
# #     """
# 
# 
# def calculate_new_elo(clubs_df: pd.DataFrame, game: pd.Series, K=30):
#     """Calculate new elo
# 
#     Args:
#         home_team_elo (_type_): _description_
#         away_team_elo (_type_): _description_
#         result (_type_): result (should be 1, 0, -1)
#         K (int, optional): _description_. Defaults to 30.
# 
#     Returns:
#         tuple (int, int): new elos of home and away team
#     """
#     result = game["is_win"]
# 
#     if result not in [-1, 0, 1]:
#         raise AssertionError("Invalid result value passed!")
# 
#     home_club_id = game["home_club_id"]
#     away_club_id = game["away_club_id"]
# 
#     home_club_elo = clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"].values[0]
#     away_club_elo = clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"].values[0]
# 
#     expected_home_team = expected_chance_of_success(home_club_elo, away_club_elo)
#     expected_away_team = 1 - expected_home_team
# 
#     new_home_elo = home_club_elo + K * (result - expected_home_team)
#     new_away_elo = away_club_elo + K * ((1 - result) - expected_away_team)
# 
#     return new_home_elo, new_away_elo
# 
# 
# def clean_games_df(games_df: pd.DataFrame, clubs_df: pd.DataFrame) -> pd.DataFrame:
#     """Clean games_df so that it only contains games between clubs that
#     exists in clubs_df
# 
#     Args:
#         games_df (pd.DataFrame): _description_
#         clubs_df (pd.DataFrame): _description_
# 
#     Returns:
#         pd.DataFrame: _description_
#     """
# 
#     # Clean games_df so that it only contains games that have valid club id (ie. Clubs in our club_df)
#     home_exists = games_df["home_club_id"].isin(clubs_df["club_id"])
#     away_exists = games_df["away_club_id"].isin(clubs_df["club_id"])
#     cleaned_games_df = games_df[home_exists & away_exists]
# 
#     return cleaned_games_df
# 
# 
# def calculate_clubs_elo_single_match(clubs_df: pd.DataFrame, row):
# 
#     home_club_id = row["home_club_id"]
#     away_club_id = row["away_club_id"]
# 
#     # Get current ELOs for home and away clubs from clubs_df
#     home_elo = clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"].values[0]
#     away_elo = clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"].values[0]
# 
#     # Get result for the game (assuming 1 for home win, 0 for away win, 0.5 for draw)
#     game_result = row["is_win"]  # Adjust this based on your game logic
# 
#     # Update ELOs based on the result
#     new_home_elo, new_away_elo = calculate_new_elo(row)
# 
#     clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"] = new_home_elo
#     clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"] = new_away_elo
# 
#     return clubs_df
# 
# 
# def calculate_clubs_elo():
#     """
#     Basically calculates clubs elo from scratch
# 
#     1. Read csv files (from transfermrkt dataset) to create a dictionary of DataFrames
#     2. Clean games and filter them by date
#     3. Calculate ELO using very basic ELO
# 
#     Returns:
#         pd.DataFrame : clubs_df with updated club ELO
#     """
# 
#     # Import data
#     dataframes = import_data_from_csv()
#     games_df = dataframes["games_df"]
#     clubs_df = dataframes["clubs_df"]
#     club_games_df = dataframes["club_games_df"]
# 
#     cleaned_games_df = clean_games_df(games_df, clubs_df)
# 
#     # Merge
#     club_games_merged_df = pd.merge(
#         cleaned_games_df, club_games_df, on="game_id", how="inner"
#     )
#     
#     club_games_merged_df.to_csv('club_games_merged.csv')
# 
#     # Now, we will sort by date (Oldest to Latest)
#     # To calculate and update ELO by time
#     # Change data type to datetime format
#     club_games_merged_df = sort_games_by_date(club_games_merged_df)
# 
#     # Initialise club elos
#     # NOTE: Here we are using 1500, but better init. can be done like considering team market value
#     # or level of league they are playing for etc
#     clubs_df["elo"] = 1500
#     # Ensure the 'elo' column is of type float
#     clubs_df["elo"] = clubs_df["elo"].astype(float)
#     test_games_df = club_games_merged_df.head(5000)
# 
#     update_counter = 0
#     for idx, row in test_games_df.iterrows():
#         # home_club_id = row["home_club_id"]
#         # away_club_id = row["away_club_id"]
# 
#         # # Get current ELOs for home and away clubs from clubs_df
#         # home_elo = clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"].values[0]
#         # away_elo = clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"].values[0]
# 
#         # # Get result for the game (assuming 1 for home win, 0 for away win, 0.5 for draw)
#         # game_result = row["is_win"]  # Adjust this based on your game logic
# 
#         # # Update ELOs based on the result
#         # new_home_elo, new_away_elo = calculate_new_elo(home_elo, away_elo, game_result)
# 
#         # clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"] = new_home_elo
#         # clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"] = new_away_elo
#         clubs_df = calculate_clubs_elo_single_match(clubs_df, row)
# 
#         update_counter += 1
# 
#     print(f"ELO was {update_counter} times updated")
# 
#     clubs_df.to_csv("club_elos.csv")
#     return clubs_df
# 
# 
# df = calculate_clubs_elo()
# print(df.head())
# print(df.tail())